###Q1

In [ ]:
# Function to evaluate symptoms and test results for diagnosis
def evaluate_condition(user_symptoms, user_tests):
    
    # Dictionary containing disease data: symptoms, tests, and treatment
    condition_data = {
        "Acute Appendicitis": {
            "symptoms": ["fever", "pain in abdomen", "vomiting"],
            "tests": ["tlc high", "dlc neutrophils high", "esr high"],
            "treatment": "Surgery"
        },
        "Pneumonia": {
            "symptoms": ["fever", "cough", "chest pain"],
            "tests": ["tlc high", "dlc neutrophils high", "esr high", "x-ray", "pneumonic patch"],
            "treatment": "Antibiotics"
        },
        "Acute Tonsillitis": {
            "symptoms": ["fever", "cough"],
            "tests": ["red enlarged tonsils", "pus in tonsils"],
            "treatment": ("Anti-allergic + Paracetamol\n"
                          "If not cured: Add antibiotics orally\n"
                          "If still not cured: Add IV antibiotics")
        }
    }
    
    # Looping through each disease and checking if it matches input criteria
    for condition, details in condition_data.items():
        # to Check if all required symptoms are present in user input
        symptoms_match = all(symptom in user_symptoms for symptom in details["symptoms"])
        # to  Check if all necessary tests are positive in user input
        tests_match = all(test in user_tests for test in details["tests"])
        
        # If both symptoms and tests match, return the condition and treatment
        if symptoms_match and tests_match:
            return condition, details["treatment"]
    
    # If no condition matches, return no diagnosis
    return None, "No diagnosis found"

# Main function to interact with the user
def run_diagnosis():
    
    #  user input for symptoms and tests, converting to lowercase
    user_symptoms = input("Enter symptoms: ").lower()  
    user_tests = input("Enter test results: ").lower()  
    
    # Calling function to evaluate the diagnosis based on input
    diagnosis, treatment_plan = evaluate_condition(user_symptoms, user_tests)
    
    # Output the diagnosis and corresponding treatment if found
    if diagnosis:
        print("Diagnosed Condition:", diagnosis)
        print("Suggested Treatment:", treatment_plan)
    else:
        print(treatment_plan)


if __name__ == "__main__":
    run_diagnosis()


Diagnosed Condition: Pneumonia
Suggested Treatment: Antibiotics


In [ ]:
#q2
import random

class WumpusExplorer:
    def __init__(self):
        # Define the Wumpus World grid environment
        self.grid = [
            ["Start", "Empty", "Empty", "Pit"],
            ["Empty", "Pit", "Wumpus", "Unknown"],
            ["Empty", "Gold", "Unknown", "Unknown"],
            ["Unknown", "Pit", "Unknown", "Unknown"]
        ]
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])
        self.start_pos = (0, 0)
        self.current_pos = self.start_pos
        self.gold_collected = False
        self.moves_made = 0
        self.max_moves = 50
        # Internal knowledge model of the world, initially unknown except for the start position
        self.knowledge_base = [['Unknown' for _ in range(self.cols)] for _ in range(self.rows)]
        self.knowledge_base[0][0] = "Start"
        # Set of visited locations to prevent redundant movement
        self.visited_locations = set()
        self.visited_locations.add(self.current_pos)

    def get_adjacent_cells(self, position):
        """Retrieve a list of valid adjacent cells (up, down, left, right)."""
        x, y = position
        adjacent_cells = []
        if x > 0:
            adjacent_cells.append((x - 1, y))
        if x < self.rows - 1:
            adjacent_cells.append((x + 1, y))
        if y > 0:
            adjacent_cells.append((x, y - 1))
        if y < self.cols - 1:
            adjacent_cells.append((x, y + 1))
        random.shuffle(adjacent_cells)  # Shuffle to introduce randomness in movement
        return adjacent_cells

    def update_knowledge(self):
        """Update internal knowledge based on adjacent percepts."""
        adjacent_cells = self.get_adjacent_cells(self.current_pos)
        for nx, ny in adjacent_cells:
            percept = self.grid[nx][ny]
            if self.knowledge_base[nx][ny] == 'Unknown':
                self.knowledge_base[nx][ny] = percept

    def is_safe_cell(self, position):
        """Determine if a cell is safe (i.e., not a Pit or Wumpus)."""
        x, y = position
        return self.knowledge_base[x][y] not in ['Pit', 'Wumpus']

    def determine_movement(self, current, target):
        """Determine movement direction based on the relative positions."""
        cx, cy = current
        nx, ny = target
        if nx < cx:
            return "Move Up"
        elif nx > cx:
            return "Move Down"
        elif ny < cy:
            return "Move Left"
        elif ny > cy:
            return "Move Right"
        else:
            return "No Movement"

    def select_next_action(self):
        """Determine the next movement strategy."""
        
        if self.gold_collected:
            # Return to the start using a simple heuristic.
            x, y = self.current_pos
            sx, sy = self.start_pos
            if x > sx and self.is_safe_cell((x - 1, y)):
                return "Move Up", (x - 1, y)
            if x < sx and self.is_safe_cell((x + 1, y)):
                return "Move Down", (x + 1, y)
            if y > sy and self.is_safe_cell((x, y - 1)):
                return "Move Left", (x, y - 1)
            if y < sy and self.is_safe_cell((x, y + 1)):
                return "Move Right", (x, y + 1)
            return "No Movement", self.current_pos

        # Prioritize exploring unvisited and safe adjacent cells.
        adjacent_cells = self.get_adjacent_cells(self.current_pos)
        unvisited_safe_cells = [cell for cell in adjacent_cells if self.is_safe_cell(cell) 
                                and cell not in self.visited_locations 
                                and self.knowledge_base[cell[0]][cell[1]] in ['Empty', 'Gold', 'Start']]
        
        if unvisited_safe_cells:
            target_pos = unvisited_safe_cells[0]
            action = self.determine_movement(self.current_pos, target_pos)
            return action, target_pos

        # If all safe neighbors are visited, move to any known safe cell.
        for target_pos in adjacent_cells:
            if self.is_safe_cell(target_pos) and self.knowledge_base[target_pos[0]][target_pos[1]] in ['Empty', 'Gold', 'Start']:
                return self.determine_movement(self.current_pos, target_pos), target_pos

        # If no safe moves exist.
        return "No Movement", self.current_pos

    def display_status(self, action):
        """Display the current status of the agent and its next move."""
        print(f"Move {self.moves_made}:")
        print(f"Current Position: {self.current_pos}")
        print("Current Knowledge of the World:")
        for row in self.knowledge_base:
            print(row)
        print(f"Planned Action: {action}")
        print("-" * 40)
        input("Press Enter to continue...\n")

    def execute(self):
        """Main loop for agent execution."""
        while self.moves_made < self.max_moves:
            self.moves_made += 1
            x, y = self.current_pos
            current_cell = self.grid[x][y]

            # Check if agent finds gold and picks it up
            if current_cell == "Gold" and not self.gold_collected:
                print(f"Gold located at {self.current_pos}!")
                print("Action: Collect Gold")
                self.gold_collected = True
                self.knowledge_base[x][y] = "Empty"  # Update knowledge since gold is collected
                self.display_status("Collect Gold")
                continue

            # Check if agent (with gold) has reached the start position
            if self.gold_collected and self.current_pos == self.start_pos:
                print("Agent has successfully returned to the start with the gold!")
                self.display_status("Success")
                return

            # Update knowledge based on current percepts
            self.update_knowledge()

            # Choose next move
            action, next_position = self.select_next_action()

            # Handle edge cases where agent is at the boundary
            if self.current_pos[0] == self.rows - 1 and action == "Move Down":
                if self.is_safe_cell((self.current_pos[0] - 1, self.current_pos[1])):
                    action = "Move Up"
                    next_position = (self.current_pos[0] - 1, self.current_pos[1])

            if action == "No Movement":
                print("No safe moves available. Agent stops execution.")
                self.display_status("No Movement")
                return

            # Execute movement and update state
            self.display_status(action)
            self.current_pos = next_position
            self.visited_locations.add(self.current_pos)

        print("Maximum move limit reached. Agent stops execution.")

# Running the agent
if __name__ == "__main__":
    explorer = WumpusExplorer()
    explorer.execute()


Move 1:
Current Position: (0, 0)
Current Knowledge of the World:
['Start', 'Empty', 'Unknown', 'Unknown']
['Empty', 'Unknown', 'Unknown', 'Unknown']
['Unknown', 'Unknown', 'Unknown', 'Unknown']
['Unknown', 'Unknown', 'Unknown', 'Unknown']
Planned Action: Move Down
----------------------------------------
Move 2:
Current Position: (1, 0)
Current Knowledge of the World:
['Start', 'Empty', 'Unknown', 'Unknown']
['Empty', 'Pit', 'Unknown', 'Unknown']
['Empty', 'Unknown', 'Unknown', 'Unknown']
['Unknown', 'Unknown', 'Unknown', 'Unknown']
Planned Action: Move Down
----------------------------------------
Move 3:
Current Position: (2, 0)
Current Knowledge of the World:
['Start', 'Empty', 'Unknown', 'Unknown']
['Empty', 'Pit', 'Unknown', 'Unknown']
['Empty', 'Gold', 'Unknown', 'Unknown']
['Unknown', 'Unknown', 'Unknown', 'Unknown']
Planned Action: Move Right
----------------------------------------
Gold located at (2, 1)!
Action: Collect Gold
Move 4:
Current Position: (2, 1)
Current Knowledge 

In [ ]:
#q3
import copy

class Game:
    def __init__(self):
        # Define the 4x4 grid with obstacles (walls)
        self.grid_layout = [
            [" ",   "C", "F",  "F"],
            ["F",   "G",  "W",  "F"],
            ["C",  "W",  "C", "G"],
            ["F",   "F",  "G",  "F"]
        ]
        self.rows = 4
        self.cols = 4
        self.player_pos = (0, 0)
        self.score = 0
        self.power_mode = False
        self.power_duration = 0

    def display_grid(self):
        # Prints the current state of the grid
        for i in range(self.rows):
            row = []
            for j in range(self.cols):
                if (i, j) == self.player_pos:
                    row.append("P")  # Represent player as 'P'
                else:
                    cell = self.grid_layout[i][j]
                    row.append(cell if cell != "W" else "#")  # Represent walls with '#'
            print("[ " + " , ".join(row) + " ]")

    def count_pellets(self):
        # Count remaining food and cherries on the grid
        food = sum(cell == "F" for row in self.grid_layout for cell in row)
        cherries = sum(cell == "C" for row in self.grid_layout for cell in row)
        return food, cherries

    def move_player(self, direction):
        r, c = self.player_pos
        new_r, new_c = r, c

        if direction == "up": new_r -= 1
        elif direction == "down": new_r += 1
        elif direction == "left": new_c -= 1
        elif direction == "right": new_c += 1
        else: return False, "Invalid move"

        # Check for out-of-bounds movement
        if not (0 <= new_r < self.rows and 0 <= new_c < self.cols):
            return False, "Boundary collision"

        # Check for wall collisions
        if self.grid_layout[new_r][new_c] == "W":
            return False, "Wall collision"

        # Check for ghost collision
        cell_content = self.grid_layout[new_r][new_c]
        if cell_content == "G" and not self.power_mode:
            self.player_pos = (new_r, new_c)
            return True, "Game Over: Ghost collision"

        # Update score and handle power mode
        if cell_content == "F":
            self.score += 5
            self.grid_layout[new_r][new_c] = " "
        elif cell_content == "C":
            self.score += 15
            self.grid_layout[new_r][new_c] = " "
            self.power_mode = True
            self.power_duration = 5  # Power mode lasts 5 moves

        # Update player position
        self.player_pos = (new_r, new_c)

        # Handle power mode timer
        if self.power_mode:
            self.power_duration -= 1
            if self.power_duration <= 0:
                self.power_mode = False

        return True, f"Moved {direction}"

    def available_moves(self):
        # Get valid movement options
        moves = []
        r, c = self.player_pos
        if r > 0 and self.grid_layout[r-1][c] != "W": moves.append("up")
        if r < self.rows-1 and self.grid_layout[r+1][c] != "W": moves.append("down")
        if c > 0 and self.grid_layout[r][c-1] != "W": moves.append("left")
        if c < self.cols-1 and self.grid_layout[r][c+1] != "W": moves.append("right")
        return moves

    def evaluate_position(self, pos):
        # Calculate the utility score for a given position
        utility = 0
        r, c = pos
        cell = self.grid_layout[r][c]
        if cell == "F": utility += 5
        elif cell == "C": utility += 15
        elif cell == "G" and self.power_mode: utility += 10

        # Penalty for ghost proximity if not in power mode
        if not self.power_mode:
            for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                nr, nc = r+dr, c+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    if self.grid_layout[nr][nc] == "G":
                        utility -= 20

        # Bonus for proximity to food or cherries
        food, cherries = self.count_pellets()
        if food + cherries == 0:
            return utility

        min_dist = float('inf')
        for i in range(self.rows):
            for j in range(self.cols):
                if self.grid_layout[i][j] in ["F", "C"]:
                    dist = abs(i - r) + abs(j - c)
                    min_dist = min(min_dist, dist)
        
        if min_dist != float('inf'):
            utility += (10 - min_dist * 2)
        
        return utility

    def choose_best_move(self):
        # Choose the best move based on utility scores
        moves = self.available_moves()
        if not moves:
            return None

        best_move = None
        highest_utility = -float('inf')
        
        for move in moves:
            r, c = self.player_pos
            if move == "up": nr, nc = r-1, c
            elif move == "down": nr, nc = r+1, c
            elif move == "left": nr, nc = r, c-1
            else: nr, nc = r, c+1
            
            utility = self.evaluate_position((nr, nc))
            
            # Avoid immediate death from ghosts
            if self.grid_layout[nr][nc] == "G" and not self.power_mode:
                utility = -1000
                
            if utility > highest_utility:
                highest_utility = utility
                best_move = move
        
        return best_move

def start_game():
    game = Game()
    move_counter = 0
    print("Initial Grid:")
    game.display_grid()

    while True:
        food, cherries = game.count_pellets()
        if food + cherries == 0:
            print("\nAll pellets consumed! Victory!")
            break

        move = game.choose_best_move()
        if not move:
            print("No valid moves left!")
            break

        print(f"\nMove {move_counter+1}:")
        success, message = game.move_player(move)
        print(f"Action: {message}")
        print(f"Position: {game.player_pos}")
        print(f"Remaining Pellets - Food: {food}, Cherries: {cherries}")
        print(f"Score: {game.score}")
        print("Current Grid:")
        game.display_grid()

        if "Game Over" in message:
            break
        move_counter += 1

if __name__ == "__main__":
    start_game()


Initial Grid:
[ P , C , F , F ]
[ F , G , # , F ]
[ C , # , C , G ]
[ F , F , G , F ]

Move 1:
Action: Moved right
Position: (0, 1)
Remaining Pellets - Food: 7, Cherries: 3
Score: 15
Current Grid:
[   , P , F , F ]
[ F , G , # , F ]
[ C , # , C , G ]
[ F , F , G , F ]

Move 2:
Action: Moved down
Position: (1, 1)
Remaining Pellets - Food: 7, Cherries: 2
Score: 15
Current Grid:
[   ,   , F , F ]
[ F , P , # , F ]
[ C , # , C , G ]
[ F , F , G , F ]

Move 3:
Action: Moved left
Position: (1, 0)
Remaining Pellets - Food: 7, Cherries: 2
Score: 20
Current Grid:
[   ,   , F , F ]
[ P , G , # , F ]
[ C , # , C , G ]
[ F , F , G , F ]

Move 4:
Action: Moved down
Position: (2, 0)
Remaining Pellets - Food: 6, Cherries: 2
Score: 35
Current Grid:
[   ,   , F , F ]
[   , G , # , F ]
[ P , # , C , G ]
[ F , F , G , F ]

Move 5:
Action: Moved down
Position: (3, 0)
Remaining Pellets - Food: 6, Cherries: 1
Score: 40
Current Grid:
[   ,   , F , F ]
[   , G , # , F ]
[   , # , C , G ]
[ P , F , G , F ]

Mo

In [ ]:
#q4

# Action costs for moving and serving rooms
move_costs = {
    "X": 5,  # Moving left + Serve Room X
    "Y": 5,  # Moving right + Serve Room Y
    "Z": 1   # Moving up + Serve Room Z
}
# no cost for returning service room

#for each move
move_descriptions = {
    "X": "Service Room -> Room X -> Service Room",
    "Y": "Service Room -> Room Y -> Service Room",
    "Z": "Service Room -> Room Z -> Service Room"
}

room_set = {"X", "Y", "Z"}  # Set of rooms to be served

# --- Depth-First Search (DFS) ___________
def depth_first_search(state, path, cost):
    # State consists of (current_position, rooms_served)
    position, rooms_served = state
    
    # The Goal condition is : all rooms served and robot back in service room
    if position == "S" and rooms_served == room_set:
        return path, cost

    # From service room, try serving any unvisited room
    if position == "S":
        for room in sorted(room_set - rooms_served):  # Sorting ensures consistent order
            step_cost = move_costs[room]
            new_state = ("S", rooms_served | {room})  # Return to service room after serving
            new_path = path + [move_descriptions[room]]
            result = depth_first_search(new_state, new_path, cost + step_cost)
            if result is not None:
                return result
    return None

# --- Depth-Limited Search (DLS) for Iterative Deepening ---
def depth_limited_search(state, path, cost, depth_limit):
    position, rooms_served = state
    
    # Goal condition
    if position == "S" and rooms_served == room_set:
        return path, cost
    
    # Stop if depth limit is reached
    if depth_limit == 0:
        return None

    # Trying serving any unvisited room from service room
    if position == "S":
        for room in sorted(room_set - rooms_served):
            step_cost = move_costs[room]
            new_state = ("S", rooms_served | {room})
            new_path = path + [move_descriptions[room]]
            result = depth_limited_search(new_state, new_path, cost + step_cost, depth_limit - 1)
            if result is not None:
                return result
    return None

# --- Iterative Deepening Depth-First Search (IDDFS) ---
def iterative_deepening_search(initial_state):
    depth = 0
    while True:
        result = depth_limited_search(initial_state, [], 0, depth)
        if result is not None:
            return result, depth
        depth += 1

# --- Main Execution ---
if __name__ == "__main__":
    start_state = ("S", frozenset())  # starting pos

    # Runing DFS
    dfs_result = depth_first_search(start_state, [], 0)
    if dfs_result:
        dfs_path, dfs_total_cost = dfs_result
        print("DFS Path:")
        print("Service Room", " -> ".join(dfs_path))
        print("DFS Total Cost:", dfs_total_cost)
    else:
        print("DFS found no solution.")

    print("\n" + "-"*50 + "\n")

    # Runing IDDFS
    iddfs_result, depth_reached = iterative_deepening_search(start_state)
    if iddfs_result:
        iddfs_path, iddfs_total_cost = iddfs_result
        print("IDDFS Path:")
        print("Service Room", " -> ".join(iddfs_path))
        print("IDDFS Total Cost:", iddfs_total_cost)
        print("Depth limit reached:", depth_reached)
    else:
        print("IDDFS found no solution.")


DFS Path:
Service Room Service Room -> Room X -> Service Room -> Service Room -> Room Y -> Service Room -> Service Room -> Room Z -> Service Room
DFS Total Cost: 11

--------------------------------------------------

IDDFS Path:
Service Room Service Room -> Room X -> Service Room -> Service Room -> Room Y -> Service Room -> Service Room -> Room Z -> Service Room
IDDFS Total Cost: 11
Depth limit reached: 3


In [ ]:
#q5

import heapq
from itertools import combinations

def generate_next_states(state, crossing_times):
    
    left_bank, flashlight_position = state
    all_tourists = set(crossing_times.keys())
    next_states = []
    
    if flashlight_position == 'A':  # to move  tourists from A to B
        for group_size in [1, 2]:
            for group in combinations(left_bank, group_size):
                cost = max(crossing_times[t] for t in group)  # Max crossing time determines cost
                new_left_bank = set(left_bank) - set(group)
                new_state = (frozenset(new_left_bank), 'B')
                action = f"Tourist {group[0]} crosses from A to B (cost {cost})" if group_size == 1 \
                         else f"Tourists {group[0]} and {group[1]} cross from A to B (cost {cost})"
                next_states.append((new_state, action, cost))
    else:  # to move  a tourist from B back to A with the flashlight
        right_bank = all_tourists - set(left_bank)
        for t in right_bank:
            cost = crossing_times[t]
            new_left_bank = set(left_bank) | {t}
            new_state = (frozenset(new_left_bank), 'A')
            action = f"Tourist {t} returns from B to A (cost {cost})"
            next_states.append((new_state, action, cost))
            
    return next_states

def uniform_cost_search(crossing_times):
    
    
    initial_state = (frozenset(crossing_times.keys()), 'A')  # All tourists start on A
    goal_state = (frozenset(), 'B')  # Goal: No tourist left on A, flashlight on B
    
    # Priority queue: (cumulative_cost, state, path_taken)
    frontier = [(0, initial_state, [])]
    explored_states = {}
    states_explored = 0
    
    while frontier:
        cost, state, path_taken = heapq.heappop(frontier)
        states_explored += 1
        
        # If the goal is reached, return the solution.
        if state == goal_state:
            return path_taken, cost, states_explored
        
        # Skip if this state has been reached before with a lower cost.
        if state in explored_states and explored_states[state] <= cost:
            continue
        explored_states[state] = cost
        
        for next_state, action, action_cost in generate_next_states(state, crossing_times):
            total_cost = cost + action_cost
            new_path = path_taken + [action]
            heapq.heappush(frontier, (total_cost, next_state, new_path))
    
    return None  # No solution found

if __name__ == "__main__":
    # Input: space-separated crossing times for tourists
    input_str = input("plz Enter  the crossing times for each tourist separated by spaces: ")
    times_list = list(map(int, input_str.split()))
    
    #  a dictionary mapping tourist IDs to crossing times
    crossing_times = {i: t for i, t in enumerate(times_list)}
    
    result = uniform_cost_search(crossing_times)
    
    if result:
        path, total_time, states_explored = result
        print("\nOptimal sequence of actions:")
        print("Initial State: All tourists on bank A, flashlight on A")
        for step, action in enumerate(path, start=1):
            print(f"Step {step}: {action}")
        print(f"\nTotal time taken for all tourists to cross the bridge: {total_time}")
        print(f"Number of states explored during search: {states_explored}")
    else:
        print("No solution found.")



Optimal sequence of actions:
Initial State: All tourists on bank A, flashlight on A
Step 1: Tourists 0 and 1 cross from A to B (cost 2)
Step 2: Tourist 0 returns from B to A (cost 1)
Step 3: Tourists 2 and 3 cross from A to B (cost 10)
Step 4: Tourist 1 returns from B to A (cost 2)
Step 5: Tourists 0 and 1 cross from A to B (cost 2)

Total time taken for all tourists to cross the bridge: 17
Number of states explored during search: 59
